In [ ]:
from pymongo import MongoClient
import pprint, dateparser

In [ ]:
atlas_URI = open("atlas_URI.txt").read().strip()

In [ ]:
client = MongoClient(atlas_URI)

#### containing exactly these two elements

In [ ]:
filter = {
    'languages': ['Korean', 'English'] # contains only these two languages, no more no less
}

#### contains at least these two elements

In [ ]:
filter = {
    'languages': {'$all': ['Korean', 'English']} # has all of these languages (can have more)
}

In [ ]:
pprint.pprint(list(client.mflix.movies.find(filter)))

#### sort, skip, limit

In [ ]:
filter = {}

In [ ]:
print(client.mflix.movies.count(filter)) # fast
# print(len(list(client.mflix.movies.find(filter)))) # slow

In [ ]:
sort_key = "tomatoes.viewer.numReviews"
page = 0
movies_per_page = 20

movies = client.mflix.movies.find(filter) \
                  .sort(sort_key, -1) # -1 is descending

In [ ]:
movies.skip(movies_per_page * page) \
               .limit(movies_per_page)

In [ ]:
(list(movies))

#### various conditional queries

In [ ]:
filters = { 'year': { '$in': [ 1995, 2005, 2015 ] } } # year is one of those three choices

In [ ]:
filters = { 'year': {'$gte': 1989, '$lt': 2000} } # 1989 <= year < 2000

In [ ]:
# find all movies except the ones which are of adult genre
filters = { 'year': { '$in': [ 1995, 2005, 2015 ] }, 
            'genre': { '$not' : {'$eq': 'Adult'} }
          }

In [ ]:
for movie in client.mflix.movies.find(filters):
    pprint.pprint(movie['title'])

#### `$exists` and `find_one`'s projection

In [ ]:
query = {"comments":{"$exists": True}}
projection = {"comments": 1}

movie = client.mflix.movies.find_one(query, projection)

pprint.pprint(movie)

#### `$elementMatch`

In [ ]:
# the query below: find movies that have comments with that name and that date condition
# (can use different comments to fit the two conditions)
query = {
  "comments.name": "Samwell Tarly",
  "comments.date": {
    "$lt": dateparser.parse("1995-01-01")
  }
}
projection = {"comments": 1}

comments = client.mflix.movies.find(query, projection).skip(1).limit(1)

pprint.pprint(list(comments))

In [ ]:
# the query below: the movie has to have a comment that is by Samwell AND was written before 1995
betterQuery = {
  "comments": {
    "$elemMatch": {
      "name": "Samwell Tarly",
      "date": {
        "$lt": dateparser.parse("1995-01-01")
      }
    }
  }
}

correctMovies = list(client.mflix.movies.find(betterQuery, projection).limit(2))

pprint.pprint(correctMovies)